In [1]:
import pandas as pd
import random
import requests
import json
import os
import time
import random



# --- Hằng số và siêu tham số Q-learning ---
LEARNING_RATE = 0.1
DISCOUNT_FACTOR = 0.9
EXPLORATION_RATE = 0.2
MOODLE_URL = 'http://localhost:8100/webservice/rest/server.php'
TOKEN = '0d902de0cea69556919f2fac50126838'  # token của local user log (custome api)

ACTIONS = [
    'read_new_resource',
    'review_old_resource',
    'attempt_new_quiz',
    'redo_failed_quiz',
    'skip_to_next_module',
    'do_quiz_harder',
    'do_quiz_easier',
    'do_quiz_same'
]

# --- Phân loại score và complete_rate thành bin (lưu mốc dưới) ---
SCORE_AVG_BINS = [0, 2, 4, 6, 8]     # 5 bin: 0-2,2-4,4-6,6-8,8-10
COMPLETE_RATE_BINS = [0.0, 0.3, 0.6] # 3 bin: 0-0.3,0.3-0.6,0.6-1.0

 # Load JSON
with open('json_course_moodle_hierarchy_clean.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Lấy tất cả sectionId_new từ lessons
section_ids = []
for section in data:
    if "lessons" in section:
        for lesson in section["lessons"]:
            section_ids.append(lesson["sectionIdNew"])
            

# --- Q-table ---
q_table = {}

# --- Hàm khởi tạo Q-table ---
def initialize_q_table(filename='q_table_results.csv'):
    initial_q_table = {}
    levels = ['easy', 'medium', 'hard']   

    for section_id in section_ids:
        for level in levels:
            for complete_rate in COMPLETE_RATE_BINS:
                for score_bin in SCORE_AVG_BINS:
                    state = (section_id, level, complete_rate, score_bin)
                    initial_q_table[state] = {action: 0.0 for action in ACTIONS}

    save_q_table_to_csv(initial_q_table, filename)
    return initial_q_table

# --- Lưu Q-table ra CSV ---
def save_q_table_to_csv(q_table_to_save, filename='q_table_results.csv'):
    q_table_data = []
    for state, actions_dict in q_table_to_save.items():
        for action, q_value in actions_dict.items():
            row = {
                'sectionid': state[0],
                'level': state[1],
                'complete_rate': state[2],
                'score_bin': state[3],
                'action': action,
                'q_value': q_value
            }
            q_table_data.append(row)
    df = pd.DataFrame(q_table_data)
    df.to_csv(filename, index=False)

# --- Load Q-table từ CSV ---
def load_q_table_from_csv(filename='q_table_results.csv'):
    q_table_loaded = {}
    if not os.path.exists(filename):
        return {}

    df = pd.read_csv(filename)
    for _, row in df.iterrows():
        state = (row['sectionid'], row['level'], row['complete_rate'], row['score_bin'])
        action = row['action']
        q_value = row['q_value']

        if state not in q_table_loaded:
            q_table_loaded[state] = {a: 0.0 for a in ACTIONS}

        q_table_loaded[state][action] = q_value
    return q_table_loaded

# --- Rời rạc hóa score 0-10 ---
def discretize_score(score):
    for b in reversed(SCORE_AVG_BINS):
        if score >= b:
            return b
    return SCORE_AVG_BINS[0]

# --- Rời rạc hóa complete rate 0-1 ---
def discretize_complete_rate(rate):
    for b in reversed(COMPLETE_RATE_BINS):
        if rate >= b:
            return b
    return COMPLETE_RATE_BINS[0]


# --- Hàm lấy Cluster của người dùng từ file CSV ---
def get_user_cluster(user_id):
    """
    Lấy thông tin cluster của người dùng từ file CSV.
    """
    file_path = "./synthetic_user_features_clustered.csv"
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print("❌ Không tìm thấy file: synthetic_user_features_clustered.csv")
        return None
    user_row = df[df['userid'] == user_id]
    if user_row.empty:
        return None
    return user_row.iloc[0]['cluster']


# --- Hàm gọi Moodle API ---
def call_api(function, extra_params):
    """
    Gọi Moodle Web Service API.
    """
    params = {
        'wstoken': TOKEN,
        'moodlewsrestformat': 'json',
        'wsfunction': function
    }
    params.update(extra_params)
    try:
        response = requests.post(MOODLE_URL, data=params)
        response.raise_for_status() # Kiểm tra lỗi HTTP
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi khi gọi Moodle API '{function}': {e}")
        return {} # Trả về dictionary rỗng để tránh lỗi

# --- Lấy Q-value ---
def get_q_value(state, action):
    global q_table
    if state not in q_table:
        q_table[state] = {a: 0.0 for a in ACTIONS}
    return q_table[state][action]

# --- Cập nhật Q-table ---
def update_q_table(state, action, reward, next_state):
    global q_table
    current_q = get_q_value(state, action)
    max_future_q = max(q_table[next_state].values()) if next_state in q_table else 0.0
    q_table[state][action] = current_q + LEARNING_RATE * (reward + DISCOUNT_FACTOR * max_future_q - current_q)

def get_reward(action, old_score, new_score, old_complete, new_complete,
               cluster=None, quiz_level='medium', passed_hard_quiz=1):
    """
    Tính reward cho từng hành động của học sinh trong một section cụ thể.
    Thêm logic cho quiz level: easy, medium, hard.
    """

    # reward cơ bản theo cải thiện điểm
    score_improvement = new_score - old_score

    # reward theo cải thiện complete rate
    complete_bonus = (new_complete - old_complete) * 10  # scale để rõ ràng

    # reward theo action
    action_bonus = 0
    if action == 'read_new_resource':
        action_bonus = 1
    elif action == 'review_old_resource':
        action_bonus = 0.5
    elif action == 'attempt_new_quiz':
        # thưởng nhiều hơn nếu quiz khó
        if quiz_level == 'easy':
            action_bonus = 1.5
        elif quiz_level == 'medium':
            action_bonus = 2
        elif quiz_level == 'hard':
            action_bonus = 3
    elif action == 'redo_failed_quiz':
        action_bonus = 2.5
    elif action == 'skip_to_next_module':
        action_bonus = -1
    elif action == 'do_quiz_harder':
        action_bonus = 3
    elif action == 'do_quiz_easier':
        action_bonus = 0.5
        if passed_hard_quiz == 0:
            action_bonus += 5  # khuyến khích chuyển sang dễ nếu không pass
    elif action == 'do_quiz_same':
        action_bonus = 1

    # reward cá nhân hóa theo cluster
    cluster_bonus = 0
    if cluster == 0: # yếu
        if action in ['review_old_resource', 'redo_failed_quiz']:
            cluster_bonus = 2
        if action == 'do_quiz_easier':
            cluster_bonus += 1  # khuyến khích học sinh yếu chuyển sang dễ
    elif cluster == 1: # mạnh, giỏi
        if action == 'do_quiz_harder':
            cluster_bonus = 2

    # reward cá nhân hóa tổng
    personal_bonus = 0

    reward = score_improvement + complete_bonus + action_bonus + cluster_bonus + personal_bonus
    return reward

# --- Chọn action ---
def suggest_next_action(current_state):
    if current_state not in q_table or random.uniform(0, 1) < EXPLORATION_RATE:
        action = random.choice(ACTIONS)
        q_value = get_q_value(current_state, action)
        return action, q_value
    else:   
        q_values = q_table[current_state]
        best_action = max(q_values, key=q_values.get)
        return best_action, q_values[best_action]

def safe_get(data, key, default=None):
    """
    Trả về giá trị từ dict hoặc list (lấy phần tử đầu nếu list).
    """
    if isinstance(data, dict):
        return data.get(key, default)
    elif isinstance(data, list) and len(data) > 0 and isinstance(data[0], dict):
        return data[0].get(key, default)
    return default


def getStateFromMoodle(userid, courseid, sectionid, type, objectid):
    # --- 1. Điểm trung bình quiz của user trong 1 section ---
    avg_quiz_score_by_section_data = call_api('local_userlog_get_user_section_avg_grade', {
        'userid': userid,
        'courseid': courseid,
        'sectionid': sectionid
    })
    avg_quiz_score_by_section = safe_get(avg_quiz_score_by_section_data, 'avg_section_grade', 0)

    # --- 3.1 Tính level ---
    if type == "quiz":
        quiz_level_data = call_api('local_userlog_get_quiz_tags', {
            'quizid': objectid,
        })
        quiz_level = safe_get(quiz_level_data, 'tag_name', 'medium')
    else:
        quiz_level = 'easy'

    # --- 3.1 Tài nguyên trong section ---
    total_resource_data = call_api('local_userlog_get_total_resources_by_section', {
        'sectionid': sectionid,
        'objecttypes[0]': 'resource',
        'objecttypes[1]': 'hvp',
    })
    total_resource = safe_get(total_resource_data, 'total_resources', 0)

    # --- 3.2 Tài nguyên đã xem ---
    viewed_resource_data = call_api('local_userlog_get_viewed_resources_distinct_by_section', {
        'userid': userid,
        'courseid': courseid,
        'sectionid': sectionid,
        'objecttypes[0]': 'resource',
        'objecttypes[1]': 'hvp',
    })
    viewed_resource = safe_get(viewed_resource_data, 'viewed_resources', 0)

    # --- 3. Kiểm tra user pass quiz gần nhất ---
    quiz_passed_data = call_api('local_userlog_get_latest_quiz_pass_status_by_section', {
        'sectionid': sectionid,
        'userid': userid,
    })
    passed_lastest_quiz = safe_get(quiz_passed_data, 'is_passed', 0) == 1

    # --- 6. Rời rạc hóa điểm và complete rate ---
    score_bin = discretize_score(avg_quiz_score_by_section) # ok
    complete_bin = discretize_complete_rate(viewed_resource/total_resource) # ok

    # --- 7. Trả về state ---
    state = (sectionid, quiz_level, complete_bin, score_bin)
    return state, passed_lastest_quiz



def follow_file(file_path):
    """
    Generator đọc dòng mới từ file (giống tail -f)
    """
    with open(file_path, "r") as f:
        # nhảy tới cuối file
        f.seek(0, os.SEEK_END)
        while True:
            line = f.readline()
            if not line:
                time.sleep(0.5)
                continue
            yield line.strip()


last_state_action = {}  # dict: userid -> (state, action)

def monitor_user_log(file_path):
    global q_table, last_state_action

    for line in follow_file(file_path):
        if not line or line.startswith("userid"):
            continue  

        try:
            userid, courseid, sectionid, type_, objectid, timestamp = line.split(",")
            userid = int(userid)
            courseid = int(courseid)
            sectionid = int(sectionid)
            objectid = int(objectid)

            print(f"\n📥 New log: user={userid}, course={courseid}, section={sectionid}, type={type_}, object={objectid}")

            # --- Lấy state hiện tại ---
            current_state, passed_quiz = getStateFromMoodle(userid, courseid, sectionid, type_, objectid)

            # --- Nếu đã có state cũ + action trước đó => update Q ---
            if userid in last_state_action:
                prev_state, prev_action = last_state_action[userid]

                reward = get_reward(
                    action=prev_action,
                    old_score=prev_state[3],
                    new_score=current_state[3],
                    old_complete=prev_state[2],
                    new_complete=current_state[2],
                    cluster=get_user_cluster(userid),
                    quiz_level=prev_state[1],
                    passed_hard_quiz=passed_quiz
                )

                print(f"🏆 Reward for user {userid}: {reward}")
                update_q_table(prev_state, prev_action, reward, current_state)
                save_q_table_to_csv(q_table)

            # --- Chọn action mới từ state hiện tại ---
            action, qv = suggest_next_action(current_state)
            print(f"💡 Suggested next action for user {userid}: {action} (Q={qv:.2f})")

            # --- Lưu lại để dùng khi có log mới ---
            last_state_action[userid] = (current_state, action)

        except Exception as e:
            print(f"❌ Error processing line {line}: {e}")
            
        

In [2]:
if __name__ == "__main__":
    q_table = load_q_table_from_csv()
    if not q_table:
        q_table = initialize_q_table()

    log_file_path = "/Users/nguyenhuuloc/Documents/MyComputer/moodledata/local_userlog_data/user_log_summary.csv"
    monitor_user_log(log_file_path)


📥 New log: user=4, course=5, section=38, type=hvp, object=1
💡 Suggested next action for user 4: do_quiz_easier (Q=0.31)

📥 New log: user=4, course=5, section=38, type=quiz_attempts, object=87
🏆 Reward for user 4: 1.5
💡 Suggested next action for user 4: do_quiz_easier (Q=0.46)


KeyboardInterrupt: 